In [50]:
#setup import libs
import numpy as np
import pandas as pd

# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [51]:
def import_file(File_Path, Local=True):
    """
    import file into DataFrame and strip leading and tailing 
    whitespaces if any.
    File_path: internet link(URL) or where file is stored
    Local: is it URL or File stored in local?, Defualt True(store in local)
    """
    import pandas as pd
    
    if Local:
        fh = open(File_Path)
    else:
        
        from urllib.request import urlopen  # the lib that handles the url stuff
        fh = urlopen(File_Path) # it's a file like object and works just like a file
        
    data = pd.read_csv(fh, header = 0) #import file as DataFrame
        
    return data

In [52]:
#import_file('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190921.txt', False)
DF_19021 = import_file('turnstile_190921.txt')
DF_19021

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,00:00:00,REGULAR,7198818,2438323
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,04:00:00,REGULAR,7198834,2438325
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,08:00:00,REGULAR,7198847,2438354
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,12:00:00,REGULAR,7198929,2438428
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,16:00:00,REGULAR,7199125,2438483
...,...,...,...,...,...,...,...,...,...,...,...
204923,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/20/2019,05:00:00,REGULAR,5554,420
204924,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/20/2019,09:00:00,REGULAR,5554,420
204925,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/20/2019,13:00:00,REGULAR,5554,420
204926,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/20/2019,17:00:00,REGULAR,5554,420


In [45]:
DF_19021.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

In [54]:
# total number of missing values 
np.count_nonzero(DF_19021.isnull())

0

## Info need to extract
__Find Stations w/ highest Entries and Exits__: But have to exclude locations w/ high tourists (compare between weekend and weekdays)  
__Narrow down which time in day has higher rate__  
__Find demographic info with Gender and industries base on zipcode?__


In [56]:
# Find Top 5 Max Exits and Entries
DF_19021.ENTRIES.max()

2129093303

In [70]:
#groupby station, find the mean for Entries and Exits
DF_19021_Station = DF_19021.groupby('STATION').agg(['mean','count'])
DF_19021_Station

ENTRIES        \
                      mean count   
STATION                            
1 AV             1.415e+08   420   
103 ST           9.585e+06   630   
103 ST-CORONA    8.163e+06   378   
104 ST           5.388e+08   378   
110 ST           6.482e+06   252   
...                    ...   ...   
WOODLAWN         6.258e+06   207   
WORLD TRADE CTR  8.880e+07   756   
WTC-CORTLANDT    2.101e+07  1470   
YORK ST          1.459e+07   126   
ZEREGA AV        3.520e+05   246   

                EXITS                                                                 \
                                                                                mean   
STATION                                                                                
1 AV                                                     1.466e+08                     
103 ST                                                   7.863e+06                     
103 ST-CORONA                                            8.149e+06                     
104 ST                                                   3.921e+08                     
110 ST                                                   6.195e+06                     
...                                                            ...                     
WOODLAWN                                                 2.427e+06                     
WORLD TRADE CTR                                          1.023e+08                     
WTC-CORTLANDT                                            9.607e+04                     
YORK ST                                                  1.168e+07                     
ZEREGA AV                                                1.903e+05                     

                       
                count  
STATION                
1 AV              420  
103 ST            630  
103 ST-CORONA     378  
104 ST            378  
110 ST            252  
...               ...  
WOODLAWN          207  
WORLD TRADE CTR   756  
WTC-CORTLANDT    1470  
YORK ST           126  
ZEREGA AV         246  

[378 rows x 4 columns]

['ENTRIES',
 'EXITS                                                               ']